In [35]:
import pandas as pd
import pickle
from scipy.stats import poisson
import math

### Leer Data

In [3]:
with open("data/dict_grupos.pkl","rb") as archivo:
    dict_grupos2022 = pickle.load(archivo)
archivo.close()

df_FIFA_Historical = pd.read_csv("data/FIFAWC.csv")
df_Qatar = pd.read_csv("data/partidosQatar.csv")

In [4]:
df_Qatar

,Local,Visitante,Marcador,Año
0,Qatar,Ecuador,Match 1,2022
1,Senegal,Netherlands,Match 2,2022
2,Qatar,Senegal,Match 18,2022
3,Netherlands,Ecuador,Match 19,2022
4,Ecuador,Senegal,Match 35,2022
...,...,...,...,...
59,Winners Match 51,Winners Match 52,Match 59,2022
60,Winners Match 57,Winners Match 58,Match 61,2022
61,Winners Match 59,Winners Match 60,Match 62,2022
62,Losers Match 61,Losers Match 62,Match 63,2022


In [5]:
df_FIFA_Historical

,Local,Visitante,Año,GolesLocal,GolesVisita,GolesTotales
0,Francia,México,1930,4,1,5
1,Argentina,Francia,1930,1,0,1
2,Chile,México,1930,3,0,3
3,Chile,Francia,1930,1,0,1
4,Argentina,México,1930,6,3,9
...,...,...,...,...,...,...
895,Rusia,Croacia,2018,2,2,4
896,Francia,Bélgica,2018,1,0,1
897,Croacia,Inglaterra,2018,2,1,3
898,Bélgica,Inglaterra,2018,2,0,2


In [18]:
## Extraer data de local y visitante por separado
df_Local = df_FIFA_Historical[["Local","GolesLocal","GolesVisita"]].copy()
df_visita = df_FIFA_Historical[["Visitante","GolesLocal","GolesVisita"]].copy()
df_Local

,Local,GolesLocal,GolesVisita
0,Francia,4,1
1,Argentina,1,0
2,Chile,3,0
3,Chile,1,0
4,Argentina,6,3
...,...,...,...
895,Rusia,2,2
896,Francia,1,0
897,Croacia,2,1
898,Bélgica,2,0


In [20]:
##3 Renombrar columnas
df_Local.rename(columns={"Local":"Equipo", "GolesLocal":"GolesAnotados","GolesVisita":"GolesRecibidos"},inplace=True)
df_visita.rename(columns={"Visitante":"Equipo", "GolesLocal":"GolesRecibidos","GolesVisita":"GolesAnotados"},inplace=True)
df_visita

,Equipo,GolesRecibidos,GolesAnotados
0,México,4,1
1,Francia,1,0
2,México,3,0
3,Francia,1,0
4,México,6,3
...,...,...,...
895,Croacia,2,2
896,Bélgica,1,0
897,Inglaterra,2,1
898,Inglaterra,2,0


In [24]:
## Concatenamos en un dataframe
equipos = pd.concat([df_Local,df_visita],ignore_index=True)

### Agrupamos por equipo y obtenemos el promedio de cada columna
fuerza_equipos = equipos.groupby(["Equipo"]).mean()
fuerza_equipos

,GolesAnotados,GolesRecibidos
Equipo,,
Alemania,2.021277,1.021277
Alemania Democrática,0.833333,0.833333
Alemania Federal,2.112903,1.241935
Angola,0.333333,0.666667
Arabia Saudita,0.687500,2.437500
...,...,...
Ucrania,1.000000,1.400000
Unión Soviética,1.709677,1.096774
Uruguay,1.553571,1.321429


In [26]:
fuerza_equipos["GolesAnotados"].index

Index(['Alemania', 'Alemania Democrática', 'Alemania Federal', 'Angola',
       'Arabia Saudita', 'Argelia', 'Argentina', 'Australia', 'Austria',
       'Bolivia', 'Bosnia y Herzegovina', 'Brasil', 'Bulgaria', 'Bélgica',
       'Camerún', 'Canadá', 'Checoslovaquia', 'Chile', 'China', 'Colombia',
       'Corea del Norte', 'Corea del Sur', 'Costa Rica', 'Costa de Marfil',
       'Croacia', 'Cuba', 'Dinamarca', 'Ecuador', 'Egipto', 'El Salvador',
       'Emiratos Árabes Unidos', 'Escocia', 'Eslovaquia', 'Eslovenia',
       'España', 'Estados Unidos', 'Francia', 'Gales', 'Ghana', 'Grecia',
       'Haití', 'Honduras', 'Hungría', 'Indias Orientales Neerlandesas',
       'Inglaterra', 'Irak', 'Irlanda', 'Irlanda del Norte', 'Irán',
       'Islandia', 'Israel', 'Italia', 'Jamaica', 'Japón', 'Kuwait',
       'Marruecos', 'México', 'Nigeria', 'Noruega', 'Nueva Zelanda', 'Panamá',
       'Paraguay', 'Países Bajos', 'Perú', 'Polonia', 'Portugal',
       'RF Yugoslavia', 'República Checa', 'Rumania

In [37]:
def Poisson(x,l):
    return (l**x * math.e**(-l))/math.factorial(x)

In [29]:
def Predecir_Puntos(local, visita):
    if local in fuerza_equipos["GolesAnotados"].index and visita in fuerza_equipos["GolesRecibidos"].index:
        lambda_local = fuerza_equipos.loc[local,"GolesAnotados"]*fuerza_equipos.loc[visita,"GolesRecibidos"]
        lambda_visita = fuerza_equipos.loc[visita,"GolesAnotados"]*fuerza_equipos.loc[local,"GolesRecibidos"]
        

Predecir_Puntos("Portugal","Uruguay")

2.158333333333333 1.8125000000000002
